# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
from pybaseball import batting_stats

In [4]:
# Batting Stats , qual is the number of players appearences 
batting = batting_stats(2007,2022, qual=200)
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
2,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246,0.609,404,0.169,0.287,NaN,NaN,NaN,11.2
6,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
46,10155,2013,Mike Trout,LAA,21,157,589,716,190,115,...,NaN,0,NaN,0,0.200,0.266,NaN,NaN,NaN,10.2
104,10155,2012,Mike Trout,LAA,20,139,559,639,182,117,...,NaN,0,NaN,0,0.221,0.293,NaN,NaN,NaN,10.1
119,9166,2012,Buster Posey,SFG,25,148,530,610,178,114,...,NaN,0,NaN,0,0.190,0.251,NaN,NaN,NaN,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5198,1698,2010,Gerald Laird,DET,30,89,270,299,56,40,...,NaN,0,NaN,0,0.166,0.252,NaN,NaN,NaN,-2.4
5368,9362,2010,Adam Moore,SEA,26,60,205,218,40,30,...,NaN,0,NaN,0,0.181,0.325,NaN,NaN,NaN,-2.4
5330,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113,0.401,282,0.174,0.316,NaN,NaN,NaN,-3.1
5027,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9


In [5]:
# Export DataFrame to csv
batting.to_csv('../Data/Raw_Data/batting.csv', index=False)

In [6]:
# Drop players with only 1 season of data 
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0]>1)

In [7]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
2,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246,0.609,404,0.169,0.287,NaN,NaN,NaN,11.2
6,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
46,10155,2013,Mike Trout,LAA,21,157,589,716,190,115,...,NaN,0,NaN,0,0.200,0.266,NaN,NaN,NaN,10.2
104,10155,2012,Mike Trout,LAA,20,139,559,639,182,117,...,NaN,0,NaN,0,0.221,0.293,NaN,NaN,NaN,10.1
119,9166,2012,Buster Posey,SFG,25,148,530,610,178,114,...,NaN,0,NaN,0,0.190,0.251,NaN,NaN,NaN,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.4
5198,1698,2010,Gerald Laird,DET,30,89,270,299,56,40,...,NaN,0,NaN,0,0.166,0.252,NaN,NaN,NaN,-2.4
5330,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113,0.401,282,0.174,0.316,NaN,NaN,NaN,-3.1
5027,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9


In [13]:
def next_season(player):
    player = player.sort_values("Season")
    player["Next_BsR"] = player["BsR"].shift(-1)
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

In [14]:
# Find the next BsR of each year 
bsr_df=batting[['Name','Season','BsR','Next_BsR']]
bsr_df

,Name,Season,BsR,Next_BsR
3693,Alfredo Amezaga,2007,0.9,2.4
3889,Alfredo Amezaga,2008,2.4,NaN
1351,Garret Anderson,2007,0.2,-2.4
2396,Garret Anderson,2008,-2.4,-0.2
3641,Garret Anderson,2009,-0.2,NaN
...,...,...,...,...
4490,Owen Miller,2022,1.0,NaN
3602,Andrew Vaughn,2021,-3.6,-3.2
2421,Andrew Vaughn,2022,-3.2,NaN
4983,Ha-seong Kim,2021,1.4,1.9


## Cleaning Data

In [15]:
null_data=batting.isnull().sum()
null_data

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
xBA         5090
xSLG        5090
xwOBA       5090
L-WAR          0
Next_BsR     963
Length: 321, dtype: int64

In [16]:
complete_cols=list(batting.columns[null_data == 0])
batting = batting[complete_cols + ["Next_BsR"]].copy()

In [17]:
# Clean DataFrame
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Soft%+,Med%+,Hard%+,Barrels,HardHit,Events,CStr%,CSW%,L-WAR,Next_BsR
3693,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,109,113,75,0,0,0,0.175,0.227,2.0,2.4
3889,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,123,111,64,0,0,0,0.178,0.244,1.2,NaN
1351,2,2007,Garret Anderson,LAA,35,108,417,450,124,76,...,104,93,109,0,0,0,0.155,0.246,1.4,-2.4
2396,2,2008,Garret Anderson,LAA,36,145,557,593,163,118,...,94,103,97,0,0,0,0.175,0.258,1.4,-0.2
3641,2,2009,Garret Anderson,ATL,37,135,496,534,133,93,...,86,104,100,0,0,0,0.190,0.274,-1.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4490,24655,2022,Owen Miller,CLE,25,130,424,472,103,70,...,131,100,83,12,106,340,0.188,0.266,-0.1,NaN
3602,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,84,99,110,35,149,321,0.185,0.285,0.2,-3.2
2421,26197,2022,Andrew Vaughn,CHW,24,134,510,555,138,92,...,94,100,104,33,203,419,0.201,0.291,0.5,NaN
4983,27506,2021,Ha-seong Kim,SDP,25,117,267,298,54,32,...,137,96,88,9,64,201,0.216,0.303,0.6,1.9


In [18]:
batting.dtypes

IDfg          int64
Season        int64
Name         object
Team         object
Age           int64
             ...   
Events        int64
CStr%       float64
CSW%        float64
L-WAR       float64
Next_BsR    float64
Length: 197, dtype: object

### Dummifying all of the categorical variables 

In [19]:
batting.dtypes[batting.dtypes == 'object']

Name       object
Team       object
Dol        object
Age Rng    object
dtype: object

In [20]:
# Drop 'Dol' (Dollar value of player) and 'Age Rng' (Player's age range during a season) 
batting = batting.drop(['Dol','Age Rng'], axis = 1, inplace = False)

In [21]:
# Assign each team name to number 
batting["team_code"] = batting["Team"].astype("category").cat.codes

In [22]:
# Copy batting data 
batting_full = batting.copy()

# Drop Nan Values from 'Next_Raw'
batting = batting.dropna().copy()

In [23]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Med%+,Hard%+,Barrels,HardHit,Events,CStr%,CSW%,L-WAR,Next_BsR,team_code
3693,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,113,75,0,0,0,0.175,0.227,2.0,2.4,11
1351,2,2007,Garret Anderson,LAA,35,108,417,450,124,76,...,93,109,0,0,0,0.155,0.246,1.4,-2.4,14
2396,2,2008,Garret Anderson,LAA,36,145,557,593,163,118,...,103,97,0,0,0,0.175,0.258,1.4,-0.2,14
2214,10,2007,David Eckstein,STL,32,117,434,484,134,108,...,108,71,0,0,0,0.210,0.236,0.7,-1.1,27
3172,10,2008,David Eckstein,- - -,33,94,324,376,86,63,...,105,69,0,0,0,0.240,0.273,-0.1,0.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,23667,2021,Wander Franco,TBR,20,70,281,308,81,51,...,106,90,12,92,245,0.143,0.217,2.5,4.0,29
4387,24618,2021,Ryan Jeffers,MIN,24,85,267,293,53,28,...,88,123,23,70,159,0.172,0.316,0.7,-1.3,18
5321,24655,2021,Owen Miller,CLE,24,60,191,202,39,27,...,111,90,8,45,138,0.180,0.296,-1.1,1.0,8
3602,26197,2021,Andrew Vaughn,CHW,23,127,417,469,98,61,...,99,110,35,149,321,0.185,0.285,0.2,-3.2,6


## Machine Learning Model to predict next BsR for batters

In [24]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rr, 
                                n_features_to_select=20, 
                                direction="forward",
                                cv=split,
                                n_jobs=8
                               )

In [26]:
# Remove some columns 
removed_columns = ["Next_BsR", "Name", "Team", "IDfg", "Season"]
selected_columns = batting.columns[~batting.columns.isin(removed_columns)]

In [27]:
# Scale the data 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
batting.loc[:,selected_columns] = scaler.fit_transform(batting[selected_columns])

In [29]:
# Most values are now between 0 and 1
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Med%+,Hard%+,Barrels,HardHit,Events,CStr%,CSW%,L-WAR,Next_BsR,team_code
count,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,...,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000,4127.000000
mean,6883.916404,2013.674340,0.398226,0.641359,0.469982,0.470404,0.397093,0.315744,0.390181,0.100526,...,0.510238,0.491598,0.111712,0.187430,0.233687,0.470054,0.555672,0.390066,-0.058347,0.470022
std,5129.967127,4.191037,0.163151,0.261221,0.243383,0.262703,0.202610,0.154141,0.169883,0.103736,...,0.135945,0.139704,0.170788,0.244534,0.295180,0.144280,0.119239,0.138384,3.307774,0.307236
min,1.000000,2007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.600000,0.000000
25%,2237.000000,2010.000000,0.260870,0.444444,0.260791,0.244810,0.225962,0.189474,0.258621,0.043478,...,0.420290,0.394161,0.000000,0.000000,0.000000,0.375566,0.475771,0.289855,-2.000000,0.187500
50%,5631.000000,2014.000000,0.391304,0.700855,0.491007,0.491349,0.399038,0.305263,0.379310,0.086957,...,0.507246,0.503650,0.000000,0.000000,0.000000,0.466063,0.555066,0.369565,-0.200000,0.468750
75%,10324.000000,2017.000000,0.521739,0.871795,0.681655,0.702422,0.562500,0.426316,0.500000,0.130435,...,0.594203,0.583942,0.190476,0.368613,0.487479,0.565611,0.638767,0.463768,1.800000,0.718750
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.100000,1.000000


In [31]:
# Fit the data 
sfs.fit(batting[selected_columns], batting["Next_BsR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=8)

In [32]:
# sfs.get_support() returns True to each column we want to select
sfs.get_support()

array([ True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False,  True, False, False, False, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,

In [33]:
# Assign sfs to a variable called predictors
predictors = list(selected_columns[sfs.get_support()])
predictors

['Age',
 'G',
 'GDP',
 'SB',
 'CS',
 'BB/K',
 'IFH%',
 'Spd',
 'WPA',
 'BsR',
 'FA% (sc)',
 'vSL (sc)',
 'CH-X (sc)',
 'CH-Z (sc)',
 'wCH/C (sc)',
 'vFA (pi)',
 'SI-X (pi)',
 'O-Swing% (pi)',
 'Pull%+',
 'Hard%+']

In [34]:
# All the seasons'years 
years = sorted(batting["Season"].unique())
years

[2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021]

In [35]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []
    
    years = sorted(data["Season"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        
        # Train data is all years before current_year
        train = data[data["Season"] < current_year]
        
        # Test data is the current_year
        test = data[data["Season"] == current_year]
        
        model.fit(train[predictors], train["Next_BsR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Next_BsR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [36]:
model_pred=backtest(batting, rr, predictors)
model_pred

,actual,prediction
4574,-1.4,-2.138257
2812,-1.7,-0.975886
2141,3.9,3.343306
605,-7.9,-5.147193
1421,-1.6,-1.938103
...,...,...
1341,4.0,2.514862
4387,-1.3,-0.840266
5321,1.0,-0.989558
3602,-3.2,-2.134378


In [37]:
# Calculate the mean squared error value (To compare the prediction from actual value)
from sklearn.metrics import mean_squared_error

mean_squared_error(model_pred["actual"], model_pred["prediction"])

5.9255496118791635

In [40]:
# Square root of mean_squared_error
5.9255496118791635 ** 0.5

2.4342451831890646

In [39]:
batting['Next_BsR'].describe()

count    4127.000000
mean       -0.058347
std         3.307774
min       -12.600000
25%        -2.000000
50%        -0.200000
75%         1.800000
max        14.100000
Name: Next_BsR, dtype: float64

std = 3.30

Square root of mean_squared_error = 2.4342451831890646 < std 
Which is good for the model

### Add player history data to improve accuracy

In [43]:
def player_history(df):
    df = df.sort_values("Season")
        
    df["player_season"] = range(0, df.shape[0])
    # Find the corr between "player_season" and "WAR"
    df["BsR_corr"] = list(df[["player_season", "BsR"]].expanding().corr().loc[(slice(None), "player_season"),"BsR"])
    df["BsR_corr"].fillna(0, inplace=True)
    
    df["BsR_diff"] = df["BsR"] / df["BsR"].shift(1)
    # Replace NaN values with 1 when there's no previous season 
    df["BsR_diff"].fillna(1, inplace=True)
    # Replace infinite value with 1
    df["BsR_diff"][df["BsR_diff"] == np.inf] = 1
    
    return df

batting = batting.groupby("IDfg", group_keys=False).apply(player_history)


In [44]:
# Find averages across seasos; tell us if a player performed better than the average of a season or no 
def group_averages(df):
    return df["BsR"] / df["BsR"].mean()

In [45]:
batting["BsR_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

In [46]:
new_predictors = predictors + ["player_season", "BsR_corr", "BsR_season", "BsR_diff"]

In [47]:
predictions = backtest(batting, rr, new_predictors)

In [48]:
# Calculate the mean squared error value (To compare the prediction from actual value)
mean_squared_error(predictions["actual"], predictions["prediction"]) 

5.748990494450231

Value slighly less than before

In [41]:
pd.Series(rr.coef_, index=new_predictors).sort_values()

Age             -2.452742
WAR             -1.622960
SLG             -1.416522
BU              -1.151857
BABIP           -1.116983
war_diff        -0.645563
SO              -0.635871
PH              -0.609808
Soft%+          -0.600335
LD+%            -0.558566
O-Swing% (sc)   -0.380682
war_corr        -0.119324
player_season    0.009694
SL%              0.305993
L-WAR            0.395193
CU-Z (sc)        0.460337
O-Contact%       0.707574
Oppo%+           0.728479
Med%             0.764690
Spd              0.788521
IBB              0.919523
SB               0.920407
Hard%+           2.776560
war_season       3.332582
dtype: float64

All small values indicated that the model is not taking these columns into account to make the prediction.

In [51]:
merged = predictions.merge(batting, left_index=True, right_index=True)
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()
next_BsR_df=merged[["IDfg", "Season", "Name", "BsR", "Next_BsR", "diff"]].sort_values(["diff"])
next_BsR_df.head()

,IDfg,Season,Name,BsR,Next_BsR,diff
2828,12775,2015,Brad Miller,0.483271,1.4,0.001271
3623,5343,2018,Brandon Crawford,0.438662,-0.9,0.001637
1207,12371,2019,Tommy La Stella,0.464684,-2.2,0.003796
4586,6609,2015,Freddy Galvis,0.631970,1.0,0.004220
3513,8001,2015,Hanley Ramirez,0.368030,-0.9,0.009411


In [54]:
import json
next_BsR_batt_js=next_BsR_df.to_json(orient = "records")
next_BsR_batt_js

'[{"IDfg":12775,"Season":2015,"Name":"Brad Miller","BsR":0.4832713755,"Next_BsR":1.4,"diff":0.0012714899},{"IDfg":5343,"Season":2018,"Name":"Brandon Crawford","BsR":0.43866171,"Next_BsR":-0.9,"diff":0.0016373113},{"IDfg":12371,"Season":2019,"Name":"Tommy La Stella","BsR":0.4646840149,"Next_BsR":-2.2,"diff":0.0037957223},{"IDfg":6609,"Season":2015,"Name":"Freddy Galvis","BsR":0.6319702602,"Next_BsR":1.0,"diff":0.0042199299},{"IDfg":8001,"Season":2015,"Name":"Hanley Ramirez","BsR":0.3680297398,"Next_BsR":-0.9,"diff":0.0094108354},{"IDfg":12546,"Season":2016,"Name":"C.J. Cron","BsR":0.3605947955,"Next_BsR":-1.4,"diff":0.009487552},{"IDfg":12532,"Season":2014,"Name":"Kolten Wong","BsR":0.6505576208,"Next_BsR":3.2,"diff":0.0097167922},{"IDfg":10306,"Season":2013,"Name":"Oswaldo Arcia","BsR":0.4646840149,"Next_BsR":-0.1,"diff":0.010021026},{"IDfg":7620,"Season":2012,"Name":"Justin Ruggiano","BsR":0.3531598513,"Next_BsR":0.0,"diff":0.0100452349},{"IDfg":6368,"Season":2018,"Name":"Adam Jones",

In [55]:
# Save JSON file in clean data file
save_file=open('../Data/Clean_Data/next_BsR_batt.js','w')
json.dump(next_BsR_batt_js, save_file, indent = 6)  
save_file.close() 